In [1]:
import os

In [2]:
%pwd

'd:\\ML_&_AI(Resources&Work)\\4.PROJECTS\\7.End To End NLP Project Implementation With Deployment Github Action- Text Summarization\\Text-Summarizer--NLP\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML_&_AI(Resources&Work)\\4.PROJECTS\\7.End To End NLP Project Implementation With Deployment Github Action- Text Summarization\\Text-Summarizer--NLP'

entity/__init__.py

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


config/configuration.py

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

config/configuration.py

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config


Create components/data_transformation.py

In [8]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

c:\Users\tushar\anaconda3\envs\text_summarizer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-01-21 03:34:30,072: INFO: config: PyTorch version 2.1.2+cu121 available.]


components/data_validation.py

In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

Create pipeline/stage_03_data_transformation.py and add try block

In [11]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-01-21 03:36:30,305: INFO: common: YAML file 'config\config.yaml' loaded successfully]
[2024-01-21 03:36:30,305: INFO: common: YAML file 'params.yaml' loaded successfully]
[2024-01-21 03:36:30,305: INFO: common: Created directory at: artifacts]
[2024-01-21 03:36:30,305: INFO: common: Created directory at: artifacts/data_transformation]


tokenizer_config.json: 100%|██████████| 88.0/88.0 [00:00<?, ?B/s]
c:\Users\tushar\anaconda3\envs\text_summarizer\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tushar\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config.json: 100%|████